In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
seed = 11667
n_collect = 200
cvs = np.concatenate([np.linspace(0, 0.2, 11), np.linspace(0.2, 0.5, 13)[1:]])
coverage0 = 0.2

folders = [
#     'dy_hi', 
#     'dy_lo',
    'dy_ladd'
]

In [3]:
def collect_fux(pars_post, coverage=coverage0, alg="VSC"):    
    m = ModelIntv()
    
    mss = list()
    mss_stats = list()

    for i, pars in enumerate(pars_post):
        y1, pars = m.find_baseline(pars, 2022)

        _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv={}, polished=True)

        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        
        ms_stats.update({'Key': i, 'Scenario': 'Baseline', 'FollowUp': 0, 'Duration': 0})
        mss_stats.append(ms_stats)
    
        for n_fu in list(range(7)): # + [24, 30, 36]:
            for dur in np.linspace(0, 4, 9):
                intv = {'FullACF': {'Coverage': coverage, 'FollowUp': n_fu, 'Duration': dur, 'ScreenAlg': alg}}
                
                _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv=intv, polished=True)
                
                ms = ms.reset_index().assign(Key=i, FollowUp=n_fu, Duration=dur)
                mss.append(ms)

                ms_stats.update({'Key': i, 'Scenario': 'Intervention', 'FollowUp': n_fu, 'Duration': dur})
                mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [4]:
for folder in folders:
    print(folder)
    for cov in [0.1, 0.2, 0.5]:
        print(folder, cov)
        
        out_path = f'out/{folder}'
        pars_post = json.load(open(f'{out_path}/Post.json', 'r'))

        sel = rd.choice(list(range(len(pars_post))), n_collect)
        sel.sort()
        pars_post = [pars_post[i] for i in sel]

        mss, mss_stats = collect_fux(pars_post, coverage=cov)
        mss_stats = mss_stats.fillna(0)
        mss.to_csv(f'{out_path}/Sim_VulFu_fudur_{cov:.2f}.csv')
        mss_stats.to_csv(f'{out_path}/Sim_VulFu_fudur_{cov:.2f}_stats.csv')

dy_ladd


In [10]:
mss_stats

,Inc,Mor,ACF_Footfall,ACF_Screened,ACF_Yield,ACF_TP,ACF_Fl,ACF_Sl,ACF_TPT,ACF_fu_Footfall,...,ACF_Alt_Yield,ACF_Alt_TP,ACF_Alt_Fl,ACF_Alt_Sl,ACF_Alt_TPT,Key,Scenario,FollowUp,Duration,Coverage
0,270088.825418,20751.059788,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Baseline,0,0.0,0.2
1,260813.656656,19556.936463,5.069395e+06,5.050549e+06,40610.282947,10800.912938,10800.912938,0.000000,29809.370009,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,0.0,0.2
2,252956.288354,18547.212688,5.069395e+06,5.050851e+06,39694.926104,10414.643501,9911.979708,502.663793,29280.282603,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,0.5,0.2
3,252963.053164,18548.131061,5.069395e+06,5.050850e+06,39695.297402,10415.074165,9912.387519,502.686646,29280.223237,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,1.0,0.2
4,252962.641365,18548.063504,5.069395e+06,5.050850e+06,39695.258454,10415.031248,9912.346724,502.684524,29280.227206,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,1.5,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,157870.028854,17138.374230,5.069395e+06,5.059422e+06,17202.455892,8692.799794,8296.614365,396.185429,8509.656098,5.412142e+06,...,0.0,0.0,0.0,0.0,0.0,19,Intervention,6,2.0,0.2
1276,157863.851707,17137.532075,5.069395e+06,5.059399e+06,17222.735811,8692.092502,8295.938595,396.153906,8530.643310,6.076006e+06,...,0.0,0.0,0.0,0.0,0.0,19,Intervention,6,2.5,0.2
1277,157861.257315,17137.329220,5.069395e+06,5.059380e+06,17239.236014,8691.845737,8295.702656,396.143081,8547.390277,6.609227e+06,...,0.0,0.0,0.0,0.0,0.0,19,Intervention,6,3.0,0.2
1278,157859.644588,17137.429796,5.069395e+06,5.059365e+06,17252.942019,8691.904300,8295.759091,396.145209,8561.037719,7.045870e+06,...,0.0,0.0,0.0,0.0,0.0,19,Intervention,6,3.5,0.2
